In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

dataset = CIFAR10(root='data/', download=True, transform=transform)
test_data = CIFAR10(root='data/', train=False, transform=transform)

val_size = 5000
train_size = len(dataset) - val_size

train_data, val_data = random_split(dataset, [train_size, val_size])

BATCH_SIZE = 10
SHUFFLE = False


train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE)

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

def show_batch(dl): 
    for images,lables in dl: 
        fig, ax = plt.subplots(figsize = (10,10))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,10).permute(1,2,0))
        break
    
# show a batch
show_batch(train_loader)
show_batch(val_loader)

In [ ]:
import torch.nn as nn

class NetLReLU(nn.Module):
    def __init__(self):
        super(NetLReLU, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.af = nn.LeakyReLU(0.1)
        self.fc1 = nn.Linear(1600, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        # Implement the forward function in the network
        x = self.conv1(x)
        x = self.af(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.af(x)
        x = x.flatten(1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.af(x)
        return x

In [ ]:
class NetTanH(nn.Module):
    def __init__(self):
        super(NetTanH, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.af = nn.Tanh()
        self.fc1 = nn.Linear(1600, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        # Implement the forward function in the network
        x = self.conv1(x)
        x = self.af(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = x.flatten(1)
        x = self.af(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.af(x)
        return x

In [ ]:
import numpy as np

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs = 10, show_plot = True, run_name = ""):
    min_loss = 10000
    # Track loss
    training_loss, validation_loss = [], []
    # Track accuracy
    training_acc, validation_acc = [], []
    print("Training started!")
    for i in range(num_epochs):
        # Track loss
        epoch_training_loss, epoch_validation_loss = 0, 0
        # track accuracy
        train_correct, val_correct = 0, 0
        # training
        model.train(True)
        print("Train true")
        for batch_nr, (data, labels) in enumerate(train_loader):
            # predict
            pred = model(data)
            # calculate accuracy
            _,preds = torch.max(pred,dim=1)
            train_correct += torch.sum(preds==labels).item()
            # Clear stored gradient values
            optimizer.zero_grad()
            loss = criterion(pred, labels)
            # Backpropagate the loss through the network to find the gradients of all parameters
            loss.backward()
            # Update the parameters along their gradients
            optimizer.step()
            # Update loss
            epoch_training_loss += loss.detach().numpy()
            print("batch nr:" , batch_nr, "len", len(train_loader))
        # validation
        print("starting validation epoch", i)
        model.eval()
        for batch_nr, (data, labels) in enumerate(val_loader):
            
            # predict
            pred = model(data)
            print("predicts val", pred)
            # calculate accuracy
            _,preds = torch.max(pred,dim=1)
            val_correct += torch.sum(preds==labels).item()
             
            # calculate loss
            loss = criterion(pred, labels)
            
            # check if loss is smaller than before, if so safe model
            if loss < min_loss:
                torch.save(model, 'best_model.pt')
                min_loss = loss
            
            # Update loss
            epoch_validation_loss += loss.detach().numpy()  
        # Save loss for plot
        training_loss.append(epoch_training_loss/train_size)
        validation_loss.append(epoch_validation_loss/val_size)
        # Save accuracy for plot
        training_acc.append(train_correct/train_size)
        validation_acc.append(val_correct/val_size)
        # Print loss every 5 epochs
        if i % 5 == 0:
            print(f'Epoch {i}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')
            print(f'Train accuracy = {train_correct/train_size}')
            print(f'Validation accuracy = {val_correct/val_size}')
        writer.add_scalar('training loss ' + run_name,
                        training_loss[-1],
                        i + 1)
        writer.add_scalar('training acc ' + run_name,
                        train_correct/train_size,
                        i + 1)
        writer.add_scalar('validation acc ' + run_name,
                        val_correct/val_size,
                        i + 1)
        #writer.add_scalar('train and validation acc ' + run_name,
        #               {train acc:training_acc[-1], val acc:validation_acc[-1]},
        #               i + 1)
        
    if show_plot:
        # Plot training and validation loss
        epoch = np.arange(len(training_loss))
        plt.figure(figsize=(8,4), dpi=100)
        plt.plot(epoch, training_loss, 'r', label='Training loss',)
        plt.plot(epoch, validation_loss, 'b', label='Validation loss')
        plt.legend()
        plt.xlabel('Epoch'), plt.ylabel('Loss')
        plt.show()
        
        # Plot training and validation accuracy
        plt.figure(figsize=(8,4), dpi=100)
        plt.plot(epoch, training_acc, 'r', label='Training accuracy',)
        plt.plot(epoch, validation_acc, 'b', label='Validation accuracy')
        plt.legend()
        plt.xlabel('Epoch'), plt.ylabel('Accuracy')
        plt.show()
        
    idx = np.argmin(validation_loss)
    print(f'lowest loss for validation set: {np.min(validation_loss)}, with an accuracy of {validation_acc[idx]}')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def test_model(model, test_loader, run_name = ""):
    y_pred, y_true = [], []
    test_acc = 0
    model.eval()
    with torch.no_grad():
        for batch_nr, (data, labels) in enumerate(test_loader):
            pred = model(data)
            _,preds = torch.max(pred,dim=1)
            test_acc += torch.sum(preds==labels).item()
            y_pred.extend(preds.numpy())
            y_true.extend(labels.numpy())

    test_acc /= len(test_loader.dataset) / 100

    print(f"Test accuracy is {np.round(test_acc)}%.") 
    # Confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cf_matrix)
    cm_display.plot(colorbar=False)
    plt.show()

In [ ]:

# Hyperparams. Set these to reasonable values
LEARNING_RATE = 0.0001
epochs = 30

# Load our network
model = NetLReLU()

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, epochs, run_name = "LReLU Adam")
model = torch.load('best_model.pt')
test_model(model, test_loader)

In [ ]:

# Hyperparams. Set these to reasonable values
LEARNING_RATE = 0.001
epochs = 30

# Load our network
model = NetLReLU()

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, epochs, run_name = "LReLU SGD")
model = torch.load('best_model.pt')
test_model(model, test_loader)

In [ ]:

# Hyperparams. Set these to reasonable values
LEARNING_RATE = 0.0001
epochs = 30 

# Load our network
model = NetTanH()

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, epochs, run_name = "tanh Adam")
model = torch.load('best_model.pt')
test_model(model, test_loader)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

writer = SummaryWriter('runs/CIFAR10')

dataiter = iter(train_loader)
images, labels = next(dataiter)

img_grid = torchvision.utils.make_grid(images)

matplotlib_imshow(img_grid)

writer.add_image('ten_CIFAR10_images', img_grid)

writer.add_graph(model, images)
writer.close()

Task 2

In [ ]:

from torchvision.models import AlexNet

LEARNING_RATE = 0.0001
epochs = 30

model = AlexNet(num_classes = 10)

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, epochs, run_name = "AlexNet")
model = torch.load('best_model.pt')
test_model(model, test_loader)
